# Data Import

In [1]:
import sys
sys.path.append(r'D:\Python\main-scripts')

import pandas as pd
from connectors import OracleConnector

# ============================================
# CONFIGURATION
# ============================================
TABLE_NAME = 'JAPAN_CURVE_RATES'
MARKET = 'DE'  # German yield curve

# ============================================
# READ DATA
# ============================================

# Connect to database
connection = OracleConnector(dsn='mopdb', user=None, pwd=None)
print("✓ Database connection established\n")

# SQL query - pull German zero yields only (excluding PAR yields)
query = f"""
SELECT 
    REFERENCE_DATE,
    PRICE_SOURCE,
    CURVE,
    ZERO_1Y,
    ZERO_2Y,
    ZERO_3Y,
    ZERO_4Y,
    ZERO_5Y,
    ZERO_6Y,
    ZERO_7Y,
    ZERO_8Y,
    ZERO_9Y,
    ZERO_10Y,
    ZERO_12Y,
    ZERO_15Y,
    ZERO_20Y,
    ZERO_25Y,
    ZERO_30Y
FROM BMI_LAB.{TABLE_NAME}
WHERE MARKET = '{MARKET}'
ORDER BY REFERENCE_DATE
"""

# Load data into pandas DataFrame
df = connection.read_sql(query)

# Print summary
print(f"✓ Loaded {len(df)} rows from {TABLE_NAME}")
print(f"✓ Market: {MARKET} (German yield curve)")
print(f"✓ Date range: {df['REFERENCE_DATE'].min()} to {df['REFERENCE_DATE'].max()}\n")

# Display first few rows
print("First 5 rows:")
print(df.head())

print("\n" + "="*80)
print("DataFrame Info:")
print("="*80)
print(df.info())

print("\n" + "="*80)
print("Sample of data:")
print("="*80)
print(df)

# Close connection
connection._cnxn.close()
print("\n✓ Database connection closed")

✓ Database connection established

✓ Loaded 26060 rows from JAPAN_CURVE_RATES
✓ Market: DE (German yield curve)
✓ Date range: 1999-01-02 00:00:00 to 2026-02-12 00:00:00

First 5 rows:
  REFERENCE_DATE PRICE_SOURCE   CURVE   ZERO_1Y   ZERO_2Y   ZERO_3Y   ZERO_4Y  \
0     1999-01-02        IBOXX  LOWESS  0.030762  0.031195  0.031997  0.032941   
1     1999-01-02        IBOXX     NSS  0.030973  0.030869  0.031764  0.033027   
2     1999-01-03        IBOXX  LOWESS  0.030762  0.031195  0.031997  0.032941   
3     1999-01-03        IBOXX     NSS  0.030973  0.030869  0.031764  0.033027   
4     1999-01-04        IBOXX  LOWESS  0.029998  0.030520  0.031295  0.032191   

    ZERO_5Y   ZERO_6Y   ZERO_7Y   ZERO_8Y   ZERO_9Y  ZERO_10Y  ZERO_12Y  \
0  0.034222  0.035681  0.037116  0.038044  0.038508  0.038488  0.038439   
1  0.034373  0.035683  0.036913  0.038051  0.039103  0.040076  0.041819   
2  0.034222  0.035681  0.037116  0.038044  0.038508  0.038488  0.038439   
3  0.034373  0.035683  0.0369

In [6]:
# ============================================
# WATERFALL SYSTEM - COMBINE YIELD CURVE DATA
# ============================================
# Priority order: BUBA LOWESS -> BUBA NSS -> BUBA FBS -> IBOXX LOWESS -> IBOXX NSS -> MTS NSS
# Only add dates not already in the dataset
# Only add rows with NO NaN values in any zero yield column

import pandas as pd
import numpy as np

# Work with a copy to keep df original
df_copy = df.copy()

# Define zero columns (excluding 40Y and 50Y as per user request)
zero_columns = ['ZERO_1Y', 'ZERO_2Y', 'ZERO_3Y', 'ZERO_4Y', 'ZERO_5Y', 
                'ZERO_6Y', 'ZERO_7Y', 'ZERO_8Y', 'ZERO_9Y', 'ZERO_10Y',
                'ZERO_12Y', 'ZERO_15Y', 'ZERO_20Y', 'ZERO_25Y', 'ZERO_30Y']

# Define priority order: (PRICE_SOURCE, CURVE)
priority_order = [
    ('IBOXX', 'LOWESS'),
    ('BUBA', 'LOWESS'),
    ('BUBA', 'NSS'),
    ('BUBA', 'FBS'),
    ('IBOXX', 'NSS'),
    ('MTS', 'NSS')
]

print("="*80)
print("WATERFALL SYSTEM - BUILDING YIELD CURVE DATASET")
print("="*80)
print("\nPriority order:")
for i, (source, curve) in enumerate(priority_order, 1):
    print(f"  {i}. {source} + {curve}")
print()

# Initialize empty dataframe for combined results
df_yieldcurve = pd.DataFrame()

# Track dates already included
included_dates = set()

# Track statistics for each source
source_stats = []

# Process each source in priority order
for i, (source, curve) in enumerate(priority_order, 1):
    print(f"\nProcessing {i}/6: {source} + {curve}...")
    print("-" * 80)
    
    # Filter for this source/curve combination
    subset = df_copy[(df_copy['PRICE_SOURCE'] == source) & (df_copy['CURVE'] == curve)].copy()
    
    if len(subset) == 0:
        print(f"  ✗ No data found for {source} + {curve}")
        source_stats.append({
            'Priority': i,
            'Source': source,
            'Curve': curve,
            'Available_Rows': 0,
            'Complete_Rows': 0,
            'New_Dates': 0,
            'Rows_Added': 0
        })
        continue
    
    print(f"  Available rows: {len(subset)}")
    print(f"  Date range: {subset['REFERENCE_DATE'].min()} to {subset['REFERENCE_DATE'].max()}")
    
    # Filter out rows with ANY NaN in zero columns
    subset_complete = subset.dropna(subset=zero_columns).copy()
    print(f"  Complete rows (no NaN): {len(subset_complete)}")
    
    if len(subset_complete) == 0:
        print(f"  ✗ No complete rows (all have NaN values)")
        source_stats.append({
            'Priority': i,
            'Source': source,
            'Curve': curve,
            'Available_Rows': len(subset),
            'Complete_Rows': 0,
            'New_Dates': 0,
            'Rows_Added': 0
        })
        continue
    
    # Filter for dates NOT already included
    new_data = subset_complete[~subset_complete['REFERENCE_DATE'].isin(included_dates)].copy()
    
    if len(new_data) == 0:
        print(f"  ✗ No new dates to add (all dates already covered)")
        source_stats.append({
            'Priority': i,
            'Source': source,
            'Curve': curve,
            'Available_Rows': len(subset),
            'Complete_Rows': len(subset_complete),
            'New_Dates': 0,
            'Rows_Added': 0
        })
        continue
    
    # Add to combined dataframe
    df_yieldcurve = pd.concat([df_yieldcurve, new_data], ignore_index=True)
    
    # Update included dates
    new_dates_added = set(new_data['REFERENCE_DATE'].unique())
    included_dates.update(new_dates_added)
    
    print(f"  ✓ Added {len(new_data)} rows covering {len(new_dates_added)} new dates")
    
    # Track statistics
    source_stats.append({
        'Priority': i,
        'Source': source,
        'Curve': curve,
        'Available_Rows': len(subset),
        'Complete_Rows': len(subset_complete),
        'New_Dates': len(new_dates_added),
        'Rows_Added': len(new_data)
    })

# Sort by date
df_yieldcurve = df_yieldcurve.sort_values('REFERENCE_DATE').reset_index(drop=True)

# ============================================
# SUMMARY STATISTICS
# ============================================

print("\n" + "="*80)
print("WATERFALL SYSTEM COMPLETE")
print("="*80)

print(f"\nTotal rows in df_yieldcurve: {len(df_yieldcurve)}")
print(f"Unique dates: {df_yieldcurve['REFERENCE_DATE'].nunique()}")
print(f"Date range: {df_yieldcurve['REFERENCE_DATE'].min()} to {df_yieldcurve['REFERENCE_DATE'].max()}")



# Verify no NaN in zero columns
nan_counts = df_yieldcurve[zero_columns].isna().sum()
total_nans = nan_counts.sum()

if total_nans == 0:
    print("\n✓✓✓ PERFECT! No NaN values in any zero yield columns! ✓✓✓")
else:
    print(f"\n✗ WARNING: Found {total_nans} NaN values in zero columns")
    print("\nNaN count by column:")
    for col in zero_columns:
        if nan_counts[col] > 0:
            print(f"  {col}: {nan_counts[col]}")

print("\n" + "="*80)
print("df_yieldcurve READY FOR USE")
print("="*80)
print(f"\nDataFrame 'df_yieldcurve' created with {len(df_yieldcurve)} rows")
print(f"Columns: {list(df_yieldcurve.columns)}")

# Display first few rows
print("\nFirst 10 rows of df_yieldcurve:")
print(df_yieldcurve.head(10))

print("\nLast 10 rows of df_yieldcurve:")
print(df_yieldcurve.tail(10))

WATERFALL SYSTEM - BUILDING YIELD CURVE DATASET

Priority order:
  1. IBOXX + LOWESS
  2. BUBA + LOWESS
  3. BUBA + NSS
  4. BUBA + FBS
  5. IBOXX + NSS
  6. MTS + NSS


Processing 1/6: IBOXX + LOWESS...
--------------------------------------------------------------------------------
  Available rows: 7350
  Date range: 1999-01-02 00:00:00 to 2026-01-26 00:00:00
  Complete rows (no NaN): 7343
  ✓ Added 7343 rows covering 7343 new dates

Processing 2/6: BUBA + LOWESS...
--------------------------------------------------------------------------------
  Available rows: 3822
  Date range: 2011-01-03 00:00:00 to 2026-01-28 00:00:00
  Complete rows (no NaN): 3821
  ✓ Added 20 rows covering 20 new dates

Processing 3/6: BUBA + NSS...
--------------------------------------------------------------------------------
  Available rows: 3820
  Date range: 2011-01-03 00:00:00 to 2026-01-28 00:00:00
  Complete rows (no NaN): 3820
  ✓ Added 2 rows covering 2 new dates

Processing 4/6: BUBA + FBS...
--

In [7]:
# ============================================
# SAVE YIELD CURVE DATA TO CSV
# ============================================

import os

# Define save path
save_path = r"\\GIMECB01\HOMEDIR-VZ$\westenb\Thesis\Data"

# Create filename using MARKET variable
filename = f"{MARKET}_YieldCurve.csv"

# Full filepath
filepath = os.path.join(save_path, filename)

# Drop source names
df_yieldcurve = df_yieldcurve.drop(columns=['PRICE_SOURCE','CURVE'])

# Save to CSV
df_yieldcurve.to_csv(filepath, index=False)

print(f"✓ Saved: {filename}")
print(f"  Location: {save_path}")
print(f"  Rows: {len(df_yieldcurve)}")

✓ Saved: DE_YieldCurve.csv
  Location: \\GIMECB01\HOMEDIR-VZ$\westenb\Thesis\Data
  Rows: 7368
